# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

08-16-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-08-17 04:20:59,33.93911,67.709953,189477,7759,NaN,NaN,Afghanistan,486.732789,4.094956
1,NaN,NaN,NaN,Albania,2022-08-17 04:20:59,41.15330,20.168300,322837,3573,NaN,NaN,Albania,11218.187504,1.106750
2,NaN,NaN,NaN,Algeria,2022-08-17 04:20:59,28.03390,1.659600,269381,6878,NaN,NaN,Algeria,614.309220,2.553261
3,NaN,NaN,NaN,Andorra,2022-08-17 04:20:59,42.50630,1.521800,45899,154,NaN,NaN,Andorra,59404.646347,0.335519
4,NaN,NaN,NaN,Angola,2022-08-17 04:20:59,-11.20270,17.873900,102636,1917,NaN,NaN,Angola,312.283707,1.867766


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,187442,187685,187966,188202,188506,188704,188820,189045,189343,189477
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,317514,317681,318638,319444,320086,320781,321345,321804,322125,322837
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,268254,268356,268478,268584,268718,268866,269008,269141,269269,269381


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7752,7753,7753,7753,7755,7755,7758,7758,7759,7759
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3563,3566,3566,3567,3568,3569,3570,3571,3571,3573
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6878,6878,6878,6878,6878,6878,6878,6878,6878,6878


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17605,17605,17605,17605,17723,17723,17723,17723,17723,17723
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,62486,62486,62486,62486,63022,63022,63022,63022,63022,63022
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6382,6382,6382,6382,6453,6453,6453,6453,6453,6453


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,221,221,221,221,222,222,222,222,222,222
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,691,691,691,691,693,693,693,693,693,693
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,101,101,101,101,101,101,101,101,101,101


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
675,1001.0,Autauga,Alabama,US,2022-08-17 04:20:59,32.539527,-86.644082,17723,222,NaN,NaN,"Autauga, Alabama, US",31722.422095,1.252610
712,1075.0,Lamar,Alabama,US,2022-08-17 04:20:59,33.779950,-88.096680,4149,66,NaN,NaN,"Lamar, Alabama, US",30054.328142,1.590745
713,1077.0,Lauderdale,Alabama,US,2022-08-17 04:20:59,34.901719,-87.656247,26718,404,NaN,NaN,"Lauderdale, Alabama, US",28812.992699,1.512089


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,93140979,1037935,0.0
India,44286256,527134,0.0
France,34455445,154282,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,93140979,1037935,0.0,92103044.0,2022-08-17 04:20:59,37.934269,-91.444083
India,44286256,527134,0.0,43759122.0,2022-08-17 04:20:59,23.088275,81.806127
France,34455445,154282,0.0,34301163.0,2022-08-17 04:20:59,6.430808,-34.730285
Brazil,34201280,681763,0.0,33519517.0,2022-08-17 04:20:59,-12.669522,-48.480493
Germany,31666475,146030,0.0,31520445.0,2022-08-17 04:20:59,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,10942596,94240,0.0
Texas,7729809,89865,0.0
Florida,6979951,78749,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,10942596,94240,0.0,10848356.0,2022-08-17 04:20:59,37.843962,-120.728594
Texas,7729809,89865,0.0,7639944.0,2022-08-17 04:20:59,31.660643,-98.653069
Florida,6979951,78749,0.0,6901202.0,2022-08-17 04:20:59,28.940755,-82.700744
New York,5915819,70745,0.0,5845074.0,2022-08-17 04:20:59,42.544151,-75.474183
Illinois,3633077,39128,0.0,3593949.0,2022-08-17 04:20:59,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3363706,32961,0.0
Florida,Miami-Dade,1423397,11438,0.0
Arizona,Maricopa,1399709,17692,0.0
Illinois,Cook,1361475,14663,0.0
Texas,Harris,1179755,11190,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3363706,32961,0.0,3330745.0,2022-08-17 04:20:59,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1423397,11438,0.0,1411959.0,2022-08-17 04:20:59,25.611236,-80.551706,12086.0
Arizona,Maricopa,1399709,17692,0.0,1382017.0,2022-08-17 04:20:59,33.348359,-112.491815,4013.0
Illinois,Cook,1361475,14663,0.0,1346812.0,2022-08-17 04:20:59,41.841448,-87.816588,17031.0
Texas,Harris,1179755,11190,0.0,1168565.0,2022-08-17 04:20:59,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,189045,321804,269141,45899,102636,11,8820,9633732,428648,9810496,...,973420,243537,11746,540222,11365784,697447,535,11903,331925,256539
2022-08-15,189343,322125,269269,45899,102636,11,8851,9633732,430361,9828745,...,975264,243562,11753,540222,11367479,697447,535,11903,332014,256544
2022-08-16,189477,322837,269381,45899,102636,11,8851,9633732,430361,9849574,...,975264,243586,11753,540681,11370462,697447,535,11903,332014,256561


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,7758,3571,6878,154,1917,0,144,129566,8637,12886,...,7423,1637,14,5778,43098,5691,0,2152,4016,5588
2022-08-15,7759,3571,6878,154,1917,0,144,129566,8643,12954,...,7429,1637,14,5778,43098,5691,0,2152,4016,5588
2022-08-16,7759,3573,6878,154,1917,0,144,129566,8643,13024,...,7429,1637,14,5779,43100,5691,0,2152,4016,5588


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/22,8/8/22,8/9/22,8/10/22,8/11/22,8/12/22,8/13/22,8/14/22,8/15/22,8/16/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17605,17605,17605,17605,17723,17723,17723,17723,17723,17723
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4100,4100,4100,4100,4149,4149,4149,4149,4149,4149
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,26436,26436,26436,26436,26718,26718,26718,26718,26718,26718
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8261,8261,8261,8261,8364,8364,8364,8364,8364,8364
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,41892,41892,41892,41892,42306,42306,42306,42306,42306,42306


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,1449812,289032,8042,2224964,917061,10901310,1613443,864915,298262,49,...,2234994,7699288,1020130,138779,22135,1993874,1760576,567426,1803137,172473
2022-08-15,1449812,289032,8042,2224964,917523,10911292,1616021,865196,299087,49,...,2237496,7721541,1020130,138779,22196,1999881,1764619,569322,1807040,172473
2022-08-16,1449812,289032,8042,2224964,918576,10942596,1616851,865798,299218,49,...,2243566,7729809,1020130,138779,22225,2002553,1764619,570094,1808702,173487


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-08-14       17723   63022    6453  7070  16209    2660   5684   35055   
2022-08-15       17723   63022    6453  7070  16209    2660   5684   35055   
2022-08-16       17723   63022    6453  7070  16209    2660   5684   35055   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-08-14         9608     5650  ...    7291   2105     9019     2137   
2022-08-15         9608     5650  ...    7291   2105     9019     2137   
2022-08-16         9608     5650  ...    7319   2114     9073     2150   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-08-14          11905  11365  6010          0     2584   1755  
2022-08-15          11905  11365  6010          0     2584   1755  
2022-08-16          11982  11439  6036          0     2598   1765  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,20026,1330,33,30901,11822,94119,13078,11164,3036,0,...,27213,89767,4940,699,122,21145,13824,7203,14980,1865
2022-08-15,20026,1330,33,30901,11822,94147,13088,11164,3036,0,...,27220,89828,4940,699,122,21163,13843,7206,14980,1865
2022-08-16,20026,1330,33,30901,11830,94240,13090,11179,3036,0,...,27225,89865,4940,699,122,21183,13843,7206,14991,1871


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-14         222     693     101  105    251      54    129     640   
2022-08-15         222     693     101  105    251      54    129     640   
2022-08-16         222     693     101  105    251      54    129     640   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-08-14          165       87  ...     148     42       81       27   
2022-08-15          165       87  ...     148     42       81       27   
2022-08-16          165       87  ...     149     42       81       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-08-14            129    16    40          0       47     20  
2022-08-15            129    16    40          0       47     20  
2022-08-16            129    16    40          0       47     20  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,0.001192,0.001428,0.000494,0.0,0.0,0.0,0.000000,0.003249,0.000000,0.001587,...,0.000000,0.000115,0.000000,0.000222,0.000126,0.0,0.0,0.0,0.000000,0.000066
2022-08-15,0.001576,0.000997,0.000476,0.0,0.0,0.0,0.003515,0.000000,0.003996,0.001860,...,0.001894,0.000103,0.000596,0.000000,0.000149,0.0,0.0,0.0,0.000268,0.000019
2022-08-16,0.000708,0.002210,0.000416,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.002119,...,0.000000,0.000098,0.000000,0.000850,0.000262,0.0,0.0,0.0,0.000000,0.000066


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,0.000000,0.00028,0.0,0.0,0.0,NaN,0.0,0.000973,0.000000,0.001866,...,0.000000,0.0,0.0,0.000519,0.000023,0.0,NaN,0.0,0.0,0.000179
2022-08-15,0.000129,0.00000,0.0,0.0,0.0,NaN,0.0,0.000000,0.000695,0.005277,...,0.000808,0.0,0.0,0.000000,0.000000,0.0,NaN,0.0,0.0,0.000000
2022-08-16,0.000000,0.00056,0.0,0.0,0.0,NaN,0.0,0.000000,0.000000,0.005404,...,0.000000,0.0,0.0,0.000173,0.000046,0.0,NaN,0.0,0.0,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,0.0,0.0,0.0,0.0,0.000477,8.000000e-07,0.000000,0.000451,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-15,0.0,0.0,0.0,0.0,0.000504,9.157000e-04,0.001598,0.000325,0.002766,0.0,...,0.001120,0.002890,0.0,0.0,0.002756,0.003013,0.002296,0.003341,0.002165,0.000000
2022-08-16,0.0,0.0,0.0,0.0,0.001148,2.869000e-03,0.000514,0.000696,0.000438,0.0,...,0.002713,0.001071,0.0,0.0,0.001306,0.001336,0.000000,0.001356,0.000920,0.005879


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-15         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-16         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...  Wyoming                                \
Admin2         Chambers Cherokee  ...     Park    Platte  Sheridan  Sublette   
2022-08-14          0.0      0.0  ...  0.00000  0.000000  0.000000  0.000000   
2022-08-15          0.0      0.0  ...  0.00000  0.000000  0.000000  0.000000   
2022-08-16          0.0      0.0  ...  0.00384  0.004275  0.005987  0.006083   

Province_State                                                                
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie    Weston  
2022-08-14       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-08-15       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-08-16       0.006468  0.006511  0.004326        NaN  0.005418  0.005698  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-15,0.0,0.0,0.0,0.0,0.000000,0.000298,0.000765,0.000000,0.0,NaN,...,0.000257,0.000679,0.0,0.0,0.0,0.000851,0.001374,0.000416,0.000000,0.000000
2022-08-16,0.0,0.0,0.0,0.0,0.000677,0.000988,0.000153,0.001344,0.0,NaN,...,0.000184,0.000412,0.0,0.0,0.0,0.000945,0.000000,0.000000,0.000734,0.003217


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-08-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-15         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-16         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                           \
Admin2         Chambers Cherokee  ...      Park Platte Sheridan Sublette   
2022-08-14          0.0      0.0  ...  0.000000    0.0      0.0      0.0   
2022-08-15          0.0      0.0  ...  0.000000    0.0      0.0      0.0   
2022-08-16          0.0      0.0  ...  0.006757    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-08-14            0.0   0.0   0.0        NaN      0.0    0.0  
2022-08-15            0.0   0.0   0.0        NaN      0.0    0.0  
2022-08-16            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,0.001065,0.001698,0.000505,0.000074,3.200000e-06,0.0,0.000182,0.001642,0.000034,0.001787,...,0.000027,0.000112,0.000288,0.000569,0.000220,0.000074,0.0,0.000006,0.000150,0.000044
2022-08-15,0.001321,0.001348,0.000490,0.000037,1.600000e-06,0.0,0.001848,0.000821,0.002015,0.001824,...,0.000961,0.000107,0.000442,0.000284,0.000185,0.000037,0.0,0.000003,0.000209,0.000032
2022-08-16,0.001014,0.001779,0.000453,0.000018,8.000000e-07,0.0,0.000924,0.000411,0.001008,0.001971,...,0.000480,0.000103,0.000221,0.000567,0.000224,0.000018,0.0,0.000002,0.000105,0.000049


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,0.000114,0.000284,2.000000e-07,1.600000e-06,2.600000e-06,NaN,0.0,0.000489,0.000002,0.003534,...,0.000017,0.0,0.0,0.000488,0.000021,0.000004,NaN,2.000000e-07,5.000000e-07,0.000103
2022-08-15,0.000121,0.000142,1.000000e-07,8.000000e-07,1.300000e-06,NaN,0.0,0.000244,0.000348,0.004406,...,0.000413,0.0,0.0,0.000244,0.000010,0.000002,NaN,1.000000e-07,2.000000e-07,0.000051
2022-08-16,0.000061,0.000351,1.000000e-07,4.000000e-07,6.000000e-07,NaN,0.0,0.000122,0.000174,0.004905,...,0.000206,0.0,0.0,0.000209,0.000029,0.000001,NaN,1.000000e-07,1.000000e-07,0.000026


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-08-14,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-15,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-16,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,0.000585,0.000246,0.000724,0.000192,0.000785,0.000508,0.000176,0.000636,0.000306,0.0,...,0.000385,0.000461,0.000291,0.000144,0.000440,0.000391,0.000419,0.000454,0.000276,0.000106
2022-08-15,0.000293,0.000123,0.000362,0.000096,0.000644,0.000712,0.000887,0.000480,0.001536,0.0,...,0.000753,0.001676,0.000146,0.000072,0.001598,0.001702,0.001357,0.001897,0.001221,0.000053
2022-08-16,0.000146,0.000061,0.000181,0.000048,0.000896,0.001790,0.000700,0.000588,0.000987,0.0,...,0.001733,0.001373,0.000073,0.000036,0.001452,0.001519,0.000679,0.001627,0.001070,0.002966


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-08-14      0.000423  0.000540  0.000701  0.000665  0.000446  0.000690   
2022-08-15      0.000211  0.000270  0.000351  0.000332  0.000223  0.000345   
2022-08-16      0.000106  0.000135  0.000175  0.000166  0.000111  0.000173   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-08-14      0.000864  0.000476  0.000716  0.000777  ...  0.000054   
2022-08-15      0.000432  0.000238  0.000358  0.000389  ...  0.000027   
2022-08-16      0.000216  0.000119  0.000179  0.000194  ...  0.001934   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-08-14      0.000097  0.000118  0.000202   0.000068  0.000085  0.000050   
2022-08-15      0.000049  0.000059  0.000101   0.000034  0.000043  0.000025   
2022-08-16      0.002162  0.003023  0.003092   0.003251  0.003277  0.002176   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-08-14      -0.503937  0.000061  0.000072  
2022-08-15      -0.503937  0.000031  0.000036  
2022-08-16      -0.503937  0.002724  0.002867  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-08-14,0.000165,0.000501,0.0,0.000060,0.000418,0.000227,0.000109,0.000047,0.000341,NaN,...,0.000273,0.000102,0.000155,0.000271,0.000066,0.000227,0.000562,0.000072,0.000123,0.000076
2022-08-15,0.000082,0.000251,0.0,0.000030,0.000209,0.000262,0.000437,0.000024,0.000170,NaN,...,0.000265,0.000391,0.000077,0.000135,0.000033,0.000539,0.000968,0.000244,0.000062,0.000038
2022-08-16,0.000041,0.000125,0.0,0.000015,0.000443,0.000625,0.000295,0.000684,0.000085,NaN,...,0.000224,0.000401,0.000039,0.000068,0.000016,0.000742,0.000484,0.000122,0.000398,0.001628


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount Bullock Butler   
2022-08-14      0.000285  0.000184  0.000005  0.0  0.000256     0.0    0.0   
2022-08-15      0.000142  0.000092  0.000002  0.0  0.000128     0.0    0.0   
2022-08-16      0.000071  0.000046  0.000001  0.0  0.000064     0.0    0.0   

Province_State                              ...   Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...      Park Platte Sheridan   
2022-08-14      0.000099      0.0      0.0  ...  0.000214    0.0     -0.0   
2022-08-15      0.000049      0.0      0.0  ...  0.000107    0.0     -0.0   
2022-08-16      0.000025      0.0      0.0  ...  0.003432    0.0     -0.0   

Province_State                                                                  
Admin2         Sublette Sweetwater Teton     Uinta Unassigned  Washakie Weston  
2022-08-14          0.0   0.000122   0.0  0.000401       -1.0  0.000340    0.0  
2022-08-15          0.0   0.000061   0.0  0.000200       -1.0  0.000170    0.0  
2022-08-16          0.0   0.000030   0.0  0.000100       -1.0  0.000085    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 